In [ ]:
import pandas as pd
from tqdm import tqdm,tgrange
import os
import gensim
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
TaggededDocument = gensim.models.doc2vec.TaggedDocument

In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# 预训练模型

## 准备预训练语料：G-06-F-17类

In [ ]:
corpus_filename = "/home/hxjiang/Pythonworkspace/patent/sample3_G-06-F-17/textual/patent_text_2007_description.xlsx"
corpus = pd.read_excel(corpus_filename, encoding='utf-8')

In [ ]:
corpus.head(5)

In [ ]:
print("number of corpus："+str(len(corpus)))

## 分词等清洗工作

In [ ]:
corpus['description'].dropna(inplace=True)
# corpus['description'] = [entry.lower() for entry in corpus['description']]

In [ ]:
corpus['description'] = [ word_tokenize(entry) for entry in corpus['description'] ]

In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


for index, entry in enumerate(tqdm(corpus['description'], ncols=60)):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]]) # 对这三类做词形还原
            Final_words.append(word_Final)
    corpus.loc[index,'description_final'] = str(Final_words)

## 将语料整理成规定的形式

In [ ]:
x_train = []

for i, text in enumerate(tqdm(corpus['description_final'], ncols=60)):
    document = TaggededDocument(text, tags=[i])
    x_train.append(document)

In [ ]:
len(x_train)

## 模型训练

In [ ]:
def train(x_train):
    doc_model = Doc2Vec(x_train, min_count=1, window=5, vector_size=300, sample=1e-3, negative=5, workers=2)
    doc_model.train(x_train, total_examples=doc_model.corpus_count, epochs=20) # corpus_count是文件个数，epochs训练次数
    return doc_model

In [ ]:
doc_model = train(x_train)

In [ ]:
doc_model.save("G-06-F-17_300d.model")

# 测试模型效果

In [ ]:
# 计算余弦相似度
def cosine_similarity(vector1, vector2):
    dot_product = 0.0
    normA = 0.0
    normB = 0.0
    for a, b in zip(vector1, vector2):
        dot_product += a * b
        normA += a ** 2
        normB += b ** 2
    if normA == 0.0 or normB == 0.0:
        return 0
    else:
        return round(dot_product / ((normA**0.5)*(normB**0.5)) * 100, 2)

In [ ]:
# 5.得到两个句子的向量后使用余弦相似度
text1 = "One embodiment of the present invention provides a system for facilitating social networking based on fashion-related information. During operation, the system receives fashion-related information from a user. Next, the system extracts the user\\\'s fashion preferences from the received information and compares the user\\\'s fashion preference with other users\\\' fashion preferences. Finally, the system groups users based on similarity of their fashion preferences."
text2 = "novel multimodal learning setting of company earnings conference call"
text1 = text1.split()
text2 = text2.split()

In [ ]:
doc_model = Doc2Vec.load("G-06-F-17_300d.model")
text1_inferred_vector = doc_model.infer_vector(text1)
doc_model = Doc2Vec.load("G-06-F-17_300d.model")
text2_inferred_vector = doc_model.infer_vector(text2)
cos = cosine_similarity(text1_inferred_vector, text2_inferred_vector)

In [ ]:
cos

# 得到文档向量

In [ ]:
load_filename = "/home/hxjiang/Pythonworkspace/patent/sample3_G-06-F-17/textual/patent_text_2010.xlsx"
load_data = pd.read_excel(load_filename, encoding='utf-8')

In [ ]:
load_data.head()

In [ ]:
print("number of corpus："+str(len(load_data['abstract'])))

## abstract

In [ ]:
load_data['abstract'].dropna(inplace=True)
# load_data['abstract'] = [entry.lower() for entry in load_data['abstract']]
load_data['abstract'] = [word_tokenize(entry) for entry in load_data['abstract']]

In [ ]:
embedding_file = "/home/hxjiang/Pythonworkspace/patent/sample3_G-06-F-17/textual/abstract_doc2vec.csv"

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index, entry in enumerate(tqdm(load_data['abstract'], ncols=60)):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    doc_model = Doc2Vec.load("G-06-F-17_300d.model")
    text_inferred_vector = doc_model.infer_vector(Final_words)
    text_inferred_vector = text_inferred_vector * 100
    vector_df = pd.DataFrame([text_inferred_vector])
    if os.path.exists(embedding_file):
        vector_df.to_csv(embedding_file, header=0, mode='a', index=False, sep=',')
    else:
        vector_df.to_csv(embedding_file, mode='a', index=False, sep=',')

## claims

In [ ]:
load_data['claims'].dropna(inplace=True)
# load_data['claims'] = [entry.lower() for entry in load_data['claims']]
load_data['claims'] = [word_tokenize(entry) for entry in load_data['claims']]

In [ ]:
embedding_file = "/home/hxjiang/Pythonworkspace/patent/sample3_G-06-F-17/textual/claims_doc2vec.csv"

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index, entry in enumerate(tqdm(load_data['claims'], ncols=60)):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    doc_model = Doc2Vec.load("G-06-F-17_300d.model")
    text_inferred_vector = doc_model.infer_vector(Final_words)
    text_inferred_vector = text_inferred_vector * 100
    vector_df = pd.DataFrame([text_inferred_vector])
    if os.path.exists(embedding_file):
        vector_df.to_csv(embedding_file, header=0, mode='a', index=False, sep=',')
    else:
        vector_df.to_csv(embedding_file, mode='a', index=False, sep=',')